In [1]:
# database.py
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import orm

### 🧩 Database setup

In [2]:
# Using SQLite (local file)
DATABASE_URL = "sqlite:///./chat_memory.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)

Base = orm.declarative_base()

### 🧩 Create a model (table for chat sessions)

In [3]:
# models.py
from sqlalchemy import Column, String, Text, DateTime
from datetime import datetime
# from .database import Base

class ChatSession(Base):
    __tablename__ = "chat_sessions"

    session_id = Column(String, primary_key=True, index=True)
    history = Column(Text)  # store messages as JSON
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)

### 🧩 Create the database table

In [4]:
# create_tables.py
# from database import Base, engine
# from models import ChatSession

Base.metadata.create_all(bind=engine)
print("✅ Database and table created successfully!")

✅ Database and table created successfully!


### 🧩 Add conversation memory management

In [5]:
# chat_memory.py
import json
import uuid
from sqlalchemy.orm import Session
from langchain.memory import ConversationBufferMemory
# from database import SessionLocal
# from models import ChatSession

In [39]:
# ---------- save memory inot sql ---------------------
def save_memory(session_id: str, memory: ConversationBufferMemory):
    db: Session = SessionLocal()

    try:
        messages = [
            {"role": "user" if msg.type=="human" else "assistant", "content": msg.content}
            for msg in memory.chat_memory.messages
        ]

        serialized = json.dumps(messages)

        db_session = db.query(ChatSession).filter_by(session_id=session_id).first()

        if db_session:
            db_session.history = serialized
        else:
            db_session = ChatSession(session_id=session_id, history=serialized)
            db.add(db_session)

        db.commit()
        print(f"✅ Memory saved for session {session_id}")
    finally:
        db.close()

In [8]:
# --- Load memory from SQL ---
def load_memory(session_id: str) -> ConversationBufferMemory | None:
    db: Session = SessionLocal()
    try:
        db_session = db.query(ChatSession).filter_by(session_id=session_id).first()
        if not db_session:
            print("❌ Session not found in DB.")
            return None

        memory = ConversationBufferMemory(return_messages=True)
        history = json.loads(db_session.history)
        for msg in history:
            if msg["role"] == "user":
                memory.chat_memory.add_user_message(msg["content"])
            else:
                memory.chat_memory.add_ai_message(msg["content"])

        print(f"✅ Loaded memory for session {session_id}")
        return memory
    finally:
        db.close()

### 🧩 Test it out

In [24]:
# test_chat.py
# from langchain.memory import ConversationStringBufferMemory
from langchain.memory import ConversationBufferMemory
# from chat_memory import save_memory, load_memory
import uuid

# step 1: create new memory

session_id = str(uuid.uuid4().hex)
memory = ConversationBufferMemory(return_messages=True)

In [ ]:
# add a chat messages
memory.chat_memory.add_user_message("Hello, How are you?")
memory.chat_memory.add_ai_message("I'm good what about you?")
memory.chat_memory.add_user_message("what is 2 + 2?")
memory.chat_memory.add_ai_message("2 +2 = 4.")

In [35]:
for msg in memory.chat_memory.messages:
    print(msg.type, ":" ,msg.content)

human : Hello, How are you?
ai : I'm good what about you?
human : what is 2 + 2?
ai : 2 +2 = 4.


In [40]:
# Step 2: Save memory
save_memory(session_id, memory)

✅ Memory saved for session de5dfbf97a1b406787bf3b2e4d9bab11


In [41]:
# Step 3: Load memory later
loaded_memory = load_memory(session_id)

✅ Loaded memory for session de5dfbf97a1b406787bf3b2e4d9bab11


In [42]:
loaded_memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello, How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm good what about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is 2 + 2?', additional_kwargs={}, response_metadata={}), AIMessage(content='2 +2 = 4.', additional_kwargs={}, response_metadata={})]), return_messages=True)

In [43]:
print("\n--- Loaded Chat History ---")
for msg in loaded_memory.chat_memory.messages:
    print(f"{msg.type.upper()}: {msg.content}")


--- Loaded Chat History ---
HUMAN: Hello, How are you?
AI: I'm good what about you?
HUMAN: what is 2 + 2?
AI: 2 +2 = 4.
